# Introduction à la régression linéaire
Jean-François Bercher


In [1]:
import matplotlib.pyplot as plt
from IPython.display import clear_output, display, HTML, Image, Javascript
from ipywidgets import interactive, fixed
import ipywidgets as widgets
import numpy as np
import pandas as pd

# Pour tracés en 3d
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d


In [2]:
%run nbinit.ipy
plt.style.use('ggplot')

... Configuring matplotlib formats
... Configuring matplotlib with inline figures
... Importing numpy as np, scipy as sp, pyplot as plt, scipy.stats as stats
   ... scipy.signal as sig
... Importing widgets, display, HTML, Image, Javascript
... Some LaTeX definitions


... Defining figures captions 


... Loading customized Javascript for interactive solutions (show/hide)


In [3]:
%%HTML
<style>
em {
    color: green;
}
strong
{
    color: blue;
}
</style>

In [4]:
# redéfinit interactive de manière à pouvoir passer des arguments
# et récupérer les résultats 
def interactive(f,*args,**kwargs):
    import ipywidgets
    def fn(*args,**kwargs):
        f(*args,**kwargs)
    w=ipywidgets.interactive(fn,*args,**kwargs)
    return w

In [5]:
%load_ext rpy2.ipython

ModuleNotFoundError: No module named 'rpy2'

# Linear Regression / Régression linéaire

La régression linéaire est un modèle très simple, *souvent inexact*, mais extrêmement efficace. En outre, les éléments que nous allons présenter ici seront réutilisés tout au long du cours, qui bâtira à la fois sur les concepts et les outils. 

Ci-dessous un exemple de fonction linéaire avec un bruit additif. Le premier problème sera de retrouver, *au mieux*, les paramètres de cette fonction.

In [ ]:
%%R -o y
L=100
y =  rnorm(L)+ 0.02*seq(1,L)
plot(y, col="red")

## Introduction - The *Advertising data* 

Ces données sont utilisées dans notre livre de référence [An Introduction to Statistical Learning
with Applications in R](http://www-bcf.usc.edu/~gareth/ISL/). Elles consistent en les nombre de  ventes d'un certain produit sur 200 marchés (villes) différent(e)s, ainsi la répartition du budget publicitaire sur chacun de ces marchés pour trois médias différents : publicité à la télévision, à la radio et dans les journaux. Bien sûr il manque Internet...

Étant donné un certain budget publicitaire, une première interrogation est de savoir si la publicité sert à quelque chose, et dans l'affermative, la question est donc de savoir comment répartir au mieux ce budget entre les trois médias afin de maximiser le nombre de ventes.. 

On doit donc s'atteler à bâtir un modèle qui puisse permettre de *prédire les ventes* en fonction de la part de budget allouée aux trois médias. 

Les portions de code ci-dessous indiquent comment récupérer ces données et présentent deux figures : d'une part les ventes brutes en fonction de leur index naturel, puis les ventes en fonction du budget publicitaire alloué à la TV. Il semble manifestement qu'il y ait un lien entre ce budget et le nombre de ventes. 

In [ ]:
##%%R -o advertising
#advertising=read.csv(file = "http://www-bcf.usc.edu/~gareth/ISL/Advertising.csv")
#summary(advertising)

In [ ]:
##%%R 
#write.csv(advertising, file = "Advertising.csv", row.names=FALSE)

Lecture avec

In [ ]:
%%R -o advertising
advertising = read.csv(file = "Advertising.csv")
plot(advertising$Sales)
plot(advertising$Sales~advertising$TV)
#$

### Ajustement d'une droite de régression à un nuage de points. 

On suppose que la relation entre la variable *dépendante* $Y$ et la variable *indépendante* $X_1$ (dite aussi *explicative* ou encore *prédicteur*) est une dépendance **linéaire**. Typiquement, on aurait un modèle de la forme
\begin{equation}
\eqboxd{
\displaystyle{
Y = \beta_0 + \beta_1 X_1+\epsilon,
}}
\end{equation}
où les $\beta_i$ sont les coefficients du modèle : $\beta_0$ est l'*intercept* et $\beta_1$ la pente --*slope*, dans le cas d'un modèle avec une seule variable indépendante $X_1$ ; $\epsilon$ comprend à la fois une erreur *irréductible* et une *erreur de modèle*. 


In [ ]:
# version Python
%matplotlib inline

def plt_line(x, y, a, b, vert_bars):
    if isinstance(x,(ndarray,list)):  x=pd.Series(x)
    if isinstance(x,(ndarray,list)):  y=pd.Series(y)        
    plt.plot(x,y,'o', color="red")
    plt.plot(x,a*x+b, color="blue")
    plt.ylim([0, 30])
    plt.ylabel("Nombre de ventes")
    plt.xlabel("Budget publicitaire (TV)")
    if vert_bars:
        for k in x.index:
            plt.plot([x[k], x[k]], [a*x[k]+b, y[k]],color="red")
    plt.text(1.1*x.max(), 0.9*y.max(), "SSE: {:3.3f}".format(np.sum((y-a*x-b)**2)), fontsize=16, 
             bbox=dict(facecolor='red', boxstyle="round,pad=0.6", edgecolor='red', alpha=0.5))
    # plt.show() # for use with interact --> https://github.com/ipython/ipython/issues/10376
    #print(vert_bars)
    
def wplt_line(val):
    a=b1_choice.value;
    b=b0_choice.value;
    vert_bars=bar.value;
    clear_output(wait=True)
    plt_line(x, y, a, b, vert_bars)

In [ ]:
from ipywidgets import Layout
x=advertising['TV']
y=advertising['Sales']

b0_choice=widgets.FloatSlider(description="Choix de l'intercept",min=0, max=10, step=0.2, value=4.8, readout_format='.3f')
b1_choice=widgets.FloatSlider(description="Choix de la pente (slope)",min=-0.1, max=0.2, step=0.005, value=0.045, readout_format='.3f')
bar=widgets.Checkbox(value=False, description="Barres verticales")
b0_choice.layout = Layout(width='90%')
b1_choice.layout = Layout(width='90%')

b0_choice.observe(wplt_line,names="value")
b1_choice.observe(wplt_line,names="value")
bar.observe(wplt_line,names="value")

#c=widgets.HBox(children=[fcw,Lw])
#d=widgets.VBox(children=[c,imprespw])
d=widgets.VBox(children=[b0_choice,b1_choice,bar])
d.box_style="info"
d.layout = Layout(width='60%', align_items='baseline', border_radius=5)
#display(d)

_=interact(plt_line,x=fixed(x), y=fixed(y), a=b1_choice, b=b0_choice, vert_bars=bar)

$\def\betab{\boldsymbol{\beta}}$
### Calcul des coefficients --  cas d'un seul prédicteur 

Un critère raisonnable pour estimer les paramètres du modèle est le critère des moindres carrés : il s'agit simplement de minimiser la somme des carrés des résidus (*residual sum of squares*) : 

\begin{align}
RSS(\betab) & = \sum_{i=1}^N  \left(y_i - \hat{y}_i\right)^2 \\ 
&=  \sum_{i=1}^N  \left(y_i- \beta_0 - \beta_1 x_{i1} \right)^2,
\end{align}

où $x_{i1}$ désigne les différentes observations de $X_1$. 

La technique de régression linéaire basée sur ces moindres carrés est souvent dénommée OLS pour *Ordinary Least Squares* dans la littérature.


Avec un seul prédicteur $X_1$, on peut dériver directement le critère (*cost function*) par rapport à $\beta_0$ et $\beta_1$ pour obtenir
$$
\beta_0 = \frac{1}{N}\sum_{i=1}^{N}y_i - \beta_1 \frac{1}{N}\sum_{i=1}^{N}x_i
= \overline{y}-\beta_1\overline{x}.
$$ 

Cette valeur dépend de $\beta_1$ qui n'est pas encore déterminé et nous pouvons maintenant écrire le critère en fonction seulement de $\beta_1$ : 
$$
RSS=\sum_{i=1}^{N} (y_i -\bar{y} - \beta_1 (x_i-\bar{x}))^2
$$
ce qui entraîne pour $\beta_1$ : 
$$ 
\eqboxa{
\beta_1=\frac{\sum_{i=1}^{N} (y_i -\bar{y})(x_i-\bar{x})}{\sum_{i=1}^{N} (x_i-\bar{x})^{2}}.
}
$$

### Remarque
Ce coefficient dépend donc du rapport de l'estimée de la covariance entre X et Y à l'estimée de la variance de X :  
$$
\beta_1=\hat{cov}(X,Y)/\hat{\sigma}_X^2, 
$$
c'est-à-dire, à un facteur près, du coefficient de corrélation entre $X$ et $Y$. **Il est ainsi instructif de constater que le coefficient de prédiction est directement lié au coefficient de corrélation entre les variables dépendante et explicative.** 
Attention au fait que cela sera moins clair (et donc moins intuitif) dans le cas d'une prédiction multivariée du fait des corrélations potentielles 
*entre* les prédicteurs. 


### Illustration en R

In [ ]:
%%R 
?lm

In [ ]:
%%R -o advertising
advertising = read.csv(file = "Advertising.csv")

In [ ]:
%%R -o lmModel
lmModel = lm(Sales ~ TV, data=advertising)
print(lmModel)

In [ ]:
%%R
plot(advertising$Sales~advertising$TV)
abline(lmModel,col="red")

In [ ]:
%%R
summary(lmModel)

Et en fonction de la Radio ?

In [ ]:
%%R 
lmModel = lm(Sales ~ Radio, data=advertising)
summary(lmModel)

In [ ]:
%%R 
lmModel = lm(Sales ~ Newspaper, data=advertising)
summary(lmModel)

## Régression linéaire -- Cas multivarié 

Utiliser une seule variable explicative peut être restrictif. Bien entendu, la variable *dépendante* $Y$  peut s'expliquer en fonction de plusieurs  variables *indépendantes* $X_1, \ldots, X_p$. On suppose à nouveau que la relation est une  dépendance **linéaire**. Le modèle prend alors la forme
$$
\eqboxd{
\displaystyle{
Y = \beta_0 + \beta_1 X_1 + \ldots \beta_p X_p + \epsilon,
}}
$$
où les $\beta_i$ sont les coefficients du modèle. Les variables indépendantes $X_i$ sont aussi appelées variables *explicatives* ou encore *prédicteurs*. 

Typiquement, on dispose de données d'*apprentissage* à partir desquelles on peut estimer les  coefficients $\beta_j$ du modèle (il s'agit donc là véritablement d'une technique *supervisée*). 

Étant donné ces *estimées* des coefficients, il est possible de *prédire* les valeurs futures par 
$$
\hat{y} =\hat{\beta}_0 + \hat{\beta}_1 x_1 + \ldots \hat{\beta}_p x_p. 
$$
$\hat{y}$ indique une prédiction faite à partir des variables $X = x$.
Le symbole $\hat{}$ dénote une valeur estimée. On pourra également évaluer la qualité du modèle en testant les performances sur une *base de test*. On appelle *résidu* la différence entre la valeur exacte et la valeur prédite :  $y_i - \hat{y}_i$.  


  $\def\betab{\mathbf{\beta}}$
Un critère raisonnable pour estimer les paramètres du modèle est le critère des moindres carrés : il s'agit simplement de minimiser la somme des carrés des résidus (*residual sum of squares*) : 
\begin{align}
RSS(\betab) & = \sum_{i=1}^N  \left(y_i - \hat{y}_i\right)^2 \\ 
&=  \sum_{i=1}^N  \left(y_i- \beta_0 - \sum_{j=1}^p \beta_j x_{ij} \right)^2.
\end{align}



$\def\yb{\mathbf{y}}\def\Xb{\mathbf{X}}$
Ce critère peut se mettre sous la forme matricielle 
$$
RSS(\betab) = \left( \yb - \Xb\betab\right)^T\left( \yb - \Xb\betab\right),
$$
où $\yb$ est un vecteur colonne de dimension $N$ et $\Xb$ une matrice de dimension $N\times(p+1)$ (la première colonne de $\Xb$ est composée de 1 de manière à prendre en compte le terme $\beta_0$ \footnote{Dans certaines fonctions de scikit-learn, cette colonne de 1 peut-être intégrée ou non}. Ce critère est une forme quadratique en les $(p+1)$ paramètres inconnus. Lorsque $\Xb$ est de rang colonne plein, on a une solution unique, qui peut être obtenue en annulant le gradient
$$
\frac{\partial RSS(\betab)}{\partial \betab} = -2 \Xb^T \left( \yb - \Xb\betab\right). 
$$
Ceci conduit donc à la solution
$$
\Xb^T \left( \yb - \Xb\betab\right) = \mathbf{0} \Rightarrow  \Xb^T \yb = \Xb^T\Xb\betab,
$$
soit 
\begin{equation} \label{eq:beta_estimates}
\eqboxd{
\hat{\betab} = \left(\Xb^T\Xb\right)^{-1} \Xb^T \yb
}
\end{equation}
On peut alors prédire $\yb$ par
$$
\label{eq:ypred}
\hat{\yb} = \Xb\hat{\betab} =  \Xb \left(\Xb^T\Xb\right)^{-1} \Xb^T \yb
$$
La matrice 
\begin{equation}
\label{eq:hatmatrix}
\eqboxb{
H = \Xb \left(\Xb^T\Xb\right)^{-1} \Xb^T }
\end{equation}

est parfois appelée la *hat matrix* dans la mesure où elle met un chapeau *hat* sur $\yb$. Vous pourrez consulter une note pas trop récente sur les moindres carrés en suivant [ce lien](Note_rapide_moindres_carrés.pdf).

- rang de $X^TX$, cas singulier, corrélation

## Application au cas des données de publicité

Et c'est là que cela devient sympathique avec R : il suffit d'utiliser la fonction `lm`

`lm` accepte un langage de formule pour spécifier le modèle à effectuer. On en trouvera une description dans cette [référence][1]. 

[1]: http://faculty.chicagobooth.edu/richard.hahn/teaching/FormulaNotation.pdf

In [ ]:
%%R 
lmModel = lm('Sales ~ Newspaper', advertising)
summary(lmModel)

La signification des coefficients est la suivante : à budget Radio fixé, l'augmentation des ventes est de ~46 unités par tranches de 1000 dollars d'investissement TV. Il semble que le budget Radio ait plus d'effet sur les ventes, mais les sommes en jeu sont plus faibles. 

In [ ]:
%%R
print(names(lmModel))
confint(lmModel)

Dès lors, il est possible de *prédire* les ventes pour une répartition données du budget publicitaire. 

In [ ]:
%%R -o coeffs
#
lmModel = lm(Sales ~ Radio+TV, data=advertising)
coeffs=lmModel$coefficients
lmModel
#$

In [ ]:
%%R
predict(lmModel,data.frame(TV=250, Radio=50),interval="prediction")

In [ ]:
%%R
predict(lmModel,data.frame(TV=c(50,150,250), Radio=c(50,150,250)),interval="confidence")

Ci-dessous une illustration de la surface de prédiction en fonction des deux variables, et des erreurs de prédiction associées. 

In [ ]:
# Create a coordinate grid
Radio = np.arange(0,50)
TV = np.arange(0,300)

B1, B2 = np.meshgrid(Radio, TV, indexing='xy')
Z = np.zeros((TV.size, Radio.size))

for (i,j),v in np.ndenumerate(Z):
        Z[i,j] =(coeffs[0] + B1[i,j]*coeffs[1] + B2[i,j]*coeffs[2])

In [ ]:
%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

# Create plot
fig = plt.figure(figsize=(8,6))
fig.suptitle('Regression: Sales ~ Radio + TV Advertising', fontsize=20)

ax = fig.add_subplot(111, projection='3d')

x = advertising.Radio
y = advertising.TV
z = advertising.Sales

ax.scatter(x, y, z, c='r', marker='o')
ax.plot_surface(B1, B2, Z, rstride=10, cstride=5, alpha=0.4)

ax.set_xlabel('Radio')
ax.set_xlim(0,50)
ax.set_ylabel('TV')
ax.set_ylim(ymin=0)
ax.set_zlabel('Sales')

plt.show()

In [ ]:
"""%matplotlib inline
# Create plot
fig = plt.figure(figsize=(8,6))
fig.suptitle('Regression: Sales ~ Radio + TV Advertising', fontsize=20)

ax = axes3d.Axes3D(fig)

ax.plot_surface(B1, B2, Z, rstride=10, cstride=5, alpha=0.4)
ax.scatter3D(advertising.Radio, advertising.TV, advertising.Sales, c='r')

#for k in x:
#     plt.plot([k, k], [a*k, y[k]],color="red")

ax.set_xlabel('Radio')
ax.set_xlim(0,50)
ax.set_ylabel('TV')
ax.set_ylim(ymin=0)
ax.set_zlabel('Sales')"""

In [ ]:
azim=widgets.FloatSlider(min=-180, max=180, step=1, value=-60)    
elev=widgets.FloatSlider(min=-180, max=360, step=1, value=30)   

#ax.set_axis_off()
@interact(elev=elev, azim=azim)
def view(elev, azim):
    ax.view_init(elev, azim)
    #ax.azim=azim
    #ax.elev=elev
    display(ax.figure)

## Évaluation des performances   

Lorsque l'on effectue une régression linéaire multiple, on doit s'intéresser aux questions suivantes : 

1. La régression est-elle utile ? En d'autres termes, au moins un des prédicteurs permet-il d'expliquer les données ?
2. Quels sont les prédicteurs utiles ? Lesquels peut-on rejeter ?
3. Quelle est la qualité de la régression ? 
4. Comment peut-on prédire de nouvelles données et quelle est la qualité de la prédiction ?

### Matrice de covariance

$\def\Var#1{\mathrm{Var}\left\{#1\right\}}$
Il est aisé de calculer la matrice de covariance de $\hat\betab$ directement à partir de (\ref{eq:beta_estimates}) : 
$$
\Var{\hat\betab} = \E{\hat\betab\hat\betab^T} = \left(\Xb^T\Xb\right)^{-1} \sigma^2
$$
(pour alléger l'écriture on a supposé que $\yb$ est centré, et donc $\hat\betab$ également). On peut également estimer la variance du bruit par 
$$
\hat{\sigma^2} = \frac{1}{N-p-1} \sum_{i=1}^N (y_i - \hat{y}_i )^2,
$$
où le $N-p-1$ correspond au nombre de degrés de liberté effectif de la variable $y_i - \hat{y}_i$, et rend l'estimateur non biaisé.  
Lorsque l'erreur de modèle $\epsilon$ peut-être considérée comme gaussienne centrée, $\epsilon \sim N(0,\sigma^2)$, alors $\hat\betab$ est gaussien vectoriel $\hat\betab \sim N\left(\mathbf{0}, \left(\Xb^T\Xb\right)^{-1} \sigma^2\right)$. L'estimée de la variance, à un facteur près, est une variable du chi2 à $(N-p-1)$ degrés de liberté. 

### RSE, R2

*  **Le RSE** : Le RSS est difficile à appréhender. On lui préfère souvent **le RSE**, *residual standard error*, qui est simplement la racine carrée de la variance estimée :

$$
RSE  =\sqrt{\hat{\sigma^2}} = \sqrt{\frac{RSS}{N-p-1} }.
$$

On pourra comprendre le RSE comme une mesure d'erreur de modèle, où d'insuffisance du modèle. 

* **Le $R^2$** : Le RSE est mesuré sur la même échelle que $Y$ (donc sensible à l'échelle de travail -- ne permet pas de comparer des modèles entre eux), et ne mesure pas le gain apporté par la modélisation. L'idée est donc de se donner une référence comme étant l'erreur en l'absence de modélisation et de normaliser les choses vis-à-vis de cette référence. 
En l'absence de variable explicative, le modèle ne comprend donc que $\beta_0$. Vous montrerez facilement que dans ce cas on obtient $\beta_0 = \bar{y}={1}/{N}\sum_{i=1}^{N}y_i$. On appelle **TSS** la somme des carrés des erreurs correspondantes (c'est-à-dire le RSS correspondant)

$$
TSS = \sum_{i=1}^N  \left(y_i - \bar{y} \right)^2. 
$$
Bien entendu le RSS pour un modèle plus complet -- par exemple avec un prédicteur, est nécessairement plus faible car on cherche alors à minimiser $\sum_{i=1}^N  \left(y_i - \beta_0 - \beta_1 x_{i1} \right)^2$ et on obtiendra le TSS dans le "pire cas" où $\beta_1=0$. En peut montrer que l'on a exactement (c'est un théorème de Pythagore)

$$
\sum_{i=1}^N  \left({y}_i - \bar{y} \right)^2  = \sum_{i=1}^N  \left({y}_i - \hat{y}_i \right)^2  + \sum_{i=1}^N  \left(\hat{y}_i - \bar{y} \right)^2, 
$$
soit

$$
\eqboxc{
TSS = RSS + ESS
}
$$
où ESS vaut pour *Explained Sum of Squares* et représente la part de la variance globale qui est expliquée par le modèle. Il ne reste plus qu'à normaliser, et on pose alors

$$
\eqboxd{
R^2 = \frac{ESS}{TSS} = 1 - \frac{RSS}{TSS}
}
$$
qui représente la proportion de la variance expliquée. 

### Intervalle de confiance

Considérons l'estimée d'un des coefficients $\beta$, disons $\hat{\beta_j}$. Cette estimée est gaussienne, de variance $v_j \sigma^2$, où $v_j$ désigne le $j^{\mathrm{e}}$ terme de la diagonale de $\left(\Xb^T\Xb\right)^{-1}$.  On peut donc construire un intervalle de confiance à $(1-2\alpha)$ selon
$$
\eqboxc{
\left(\hat{\beta_j} - z^{(1-\alpha)} \sqrt{v_j} \sigma, \hat{\beta_j} + z^{(1-\alpha)} \sqrt{v_j} \sigma\right)
}
$$
où $z^{(1-\alpha)}$ est le $(1-\alpha)$ quantile d'une gaussienne normalisée. Typiquement, pour $\alpha=0.025$, on a $z^{(1-\alpha)}=1.96$, pour $\alpha=0.05$, on a $z^{(1-\alpha)}=1.645$. Pour un intervalle de confiance à 95%, on a donc approximativement 95% de chance que l'intervalle 
$$
\eqboxc{
\left(\hat{\beta_j} - 2 \sqrt{v_j} \sigma, \hat{\beta_j} + 2\sqrt{v_j} \sigma\right)
}
$$
contienne la vraie valeur $\beta_j$. Attention au fait que l'intervalle de confiance présenté ci-dessus en imaginant que l'on connait $\sigma^2$, ce qui n'est pas le cas. En réalité, on remplace $\sigma^2$ par son estimée et il faudrait rechercher les quantiles d'une variable de student plutôt que d'une gaussienne. 

### Test d'hypothèses, p-value 

La théorie des tests permet de tester ici si un coefficient de prédiction $\beta_j$ existe ou non. On considère classiquement les deux hypothèses 

- $H_0$ : $\beta_j=0$ (« pas de relation entre $Y$ et $X_j$ »)
- $H_1$ : $\beta_j\neq 0$

Pour cela, on compare la valeur estimée à son écart-type (lui aussi estimé). Grossièrement, si $\beta_j$ est de l'ordre de grandeur ou plus grand que l'écart-type, on aura tendance à rejeter l'hypothèse selon laquelle $\beta_j=0$. Lorsque $\beta_j$ est inférieur à son écart-type, on ne pourra pas rejeter la possibilité $\beta_j=0$.

On définit un score normalisé par 
$$
t_j = \frac{\hat{\beta_j}}{\sqrt{v_j} \hat{\sigma}}. 
$$
Sous l'hypothèse $H_0$, c'est-à-dire si le véritable $\beta_j$ est nul, alors $\hat{\beta_j}$ et $\hat{\sigma}$ étant respectivement gaussien et du chi-2 à $(N−p−1)$ degrés de liberté, leur rapport est une [variable de Student](https://fr.wikipedia.org/wiki/Loi_de_Student) à $(N−p−1)$ degrés de liberté. La probabilité qu'une telle variable soit égale ou supérieure à la valeur calculée $|t_j|$  est appelée la *p-value* :
$$
p_j = \mathrm{Pr}\left(X \geq |t_j| \right).
$$
Cette *p-value* peut être lue dans une table ou calculée. Elle s'interprète de la manière suivante : 

- *une faible valeur* indique qu'il est très peu probable d'observer la valeur courante ou supérieure sous $H_0$. En d'autres termes on peut rejeter $H_0$ et inférer que $\beta_j$ est effectivement non-nul.
- *une forte valeur* indique que la probabilité d'obtenir cette valeur sous l'hypothèse nulle est importante et qu'on ne peut donc pas exclure $\beta_j=0$.

### La F-statistic

Celle-ci procède de la même idée : procéder à un test d'hypothèse entre deux situations. Typiquement, on compare un modèle "0" avec $p_0+1$ coefficients, à un modèle "1" avec $p_1+1$ coefficients, $p_1>p_0$. On note RSS0 et RSS1 les RSS correspondants. On compare ensuite ces deux RSS, normalisés par rapport à RSS1, selon
$$
F = \frac{\left(RSS0-RSS1\right)/(p_1-p_0)}{RSS1/(N-p_1-1)}
$$
Sous l'hypothèse de résidus gaussiens et sous l'hypothèse H0 (selon laquelle le modèle avec le moins de coefficients est correct), $F$ suit une distribution de Fisher de paramètres $p_1-p_0$ et $N-p_1-1$. Il est alors possible de calculer cette statistique (plus $F$ est élevé et plus on va avoir tendance à rejeter H0 et accepter que le modèle à $p_1$ coefficients apporte quelque chose par rapport à celui à $p_0$ coefficients) ou encore la $p$-value correspondante (plus $p$ est faible plus on peut rejeter $H_0$). 

On utilisera souvent ce test pour *mesurer si la régression linéaire est utile*. On teste donc si par rapport au modèle de base *baseline* (qui ne comprend que le coefficient $\beta_0$), un moèle à $p$ coefficients permet de baisser sustantiellement le RSS. On a ainsi

- $H_0$ : $\beta_1=\beta_2=\ldots=\beta_p=0$ (« pas de relation entre $Y$ et les $X_j$ »)
- $H_1$ : au moins un des $\beta_j\neq 0$.

La statistique est alors
$$
F = \frac{\left(TSS-RSS\right)/(p)}{RSS/(N-p-1)}
$$

## Les données de publicité (advertising)

In [ ]:
%%R 
# On examine quelles sont les corrélations 
cor(advertising[2:5])


In [ ]:
%%R 
#install.packages("corrplot")

In [ ]:
%%R
library(corrplot)
c=cor(advertising[2:5])
corrplot(c, method="color",  addCoef.col="black", addCoefasPercent = TRUE,)

In [ ]:
%%R -o out
out=summary(lmModel)
out

## Sélection des prédicteurs

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

est = smf.ols('Sales ~ TV ', advertising).fit()
print(est.summary().tables[0])
print(est.summary().tables[1])

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

est = smf.ols('Sales ~ TV + Radio', advertising).fit()
print(est.summary().tables[0])
print(est.summary().tables[1])

En passant de la prédiction TV à TV+Radio, on constate que le R2 augmente, la p-value de la F-statistique diminue (le modèle est significatif). 

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

est = smf.ols('Sales ~ TV + Radio+Newspaper', advertising).fit()
print(est.summary().tables[0])
print(est.summary().tables[1])

Ainsi, alorsque le coefficient pour *Newspaper* était tout-à-fait significatif pour une régression linéaire simple, dans le modèle multivarié, le coefficient devient très faible et même inférieur à son écart type. Le R2 n'a pas progressé. La p-value correspondante indique que le coefficient obtenu n'est pas significatif... 

Comment est-ce possible ? Lorsqu'on observe les corrélations, on observe une corrélation de 35% entre Radio et Newspaper. Ainsi, lorsque Radio augmente, Newspaper également. Maintenant, quand on investit sur la Radio, les ventes augmentent. Donc *il semble* que les newspapers aient une influence sur les ventes alors qu'il est possible que ce ne soit qu'une conséquence de la corrélation : augmenter la radio augmente les ventes et le budget en newspaper, mais newspaper n'influe pas sur les ventes... Une illustration du célèbre 
**Corrélation n'est pas causalité**

Illustration de la corrélation entre Journaux et Radio : peut-on prédire l'un en fonction de l'autre ?

In [ ]:
%%R 
news_radio = lm('Newspaper ~  Radio', advertising)
plot(advertising$Newspaper~advertising$Radio)
abline(news_radio,col="red")
summary(news_radio)

**Sélection des variables explicatives**

Une question importante donc d'examiner quelles sont les variables à prendre en compte; Nous n'allons faire ici qu'effleurer le sujet car un cours ultérieur abordera plus précisément cette question. Pour bien poser les choses, il faut comprendre que plus on ajoute de variables, plus le RSE diminue et plus le R2 augmente. Il ne semble donc pas y avoir de problème, sauf que le modèle des moindres carrés va aller chercher des infos dans les variables même lorsqu'elles sont insensées. Ci-dessous un petit exemple :

In [ ]:
%%R
# On ajoute six colonnes de bruit
advertising$extra1 = rnorm(200)
advertising$extra2 = rnorm(200)
advertising$extra3 = rnorm(200)
advertising$extra4 = rnorm(200)
advertising$extra5 = rnorm(200)
advertising$extra6 = rnorm(200)
advertising$extra7 = rnorm(200)
advertising$extra8 = rnorm(200)
advertising$extra9 = rnorm(200)
advertising$extra10 = rnorm(200)
#
a=c('TV','Radio','Newspaper', 'extra1', 'extra2', 'extra3', 'extra4', 'extra5', 'extra6', 'extra7', 'extra8', 'extra9', 'extra10')
sformula="Sales~"
rsq=vector(); arsq=vector(); predmse=vector();
ind = sample(seq_len(200), size = 150)
train <- advertising[ind, ]
test <- advertising[-ind, ]

for (k in seq(13)){
    sformula=paste(sformula,a[k],sep="+")
    formula=as.formula(sformula)
    res= lm(formula, data=train)
    #names(summary(res))    
    rsq[k]=summary(res)$r.squared  # R-squared
    arsq[k]=summary(res)$adj.r.squared #Adjusted R-squared
    # Prediction on test
    yhat = predict(res, newdata=test)
    err = yhat-test$Sales
    predmse[k] = mean(err**2)
    #
    print(paste(a[k],"--", summary(res)$r.squared, '--', predmse[k]))
    }
x=seq(13)
#plot(rsq, type='b', xlab='', xaxt='n', ylim=c(0.88, 0.91))
plot(predmse, type='b', xlab='', xaxt='n', ylim=c(2, 4))
axis(1, at=x, labels=a, las=2)
#lines(arsq, type='b', ylim=c(0.88, 0.91))

In [ ]:
%%R
yhat = predict(res, newdata=test)
err = yhat-test$Sales
mean(err**2)

C'est sur les données nouvelles que les choses se gâtent. En effet, si on a surparamétrisé, -- on appelle cela de l'*overfitting*,  les performances de prédiction sur de nouvelles données se dégradent, et il existe quelque part un ordre optimal : ne pas prendre trop de prédicteurs pour rester stable, mais assez pour approcher au mieux la vérité (qui est inconnue !). On voit dès maintenant qu'il sera nécessaire d'évaluer les résultats sur une base de test indépendante des données d'apprentissage. On reverra cela plus longuement ultérieurement. 

À partir des données d'apprentissage, on peut penser utiliser directement les $p$-values de chacun des coefficients, accepter les variables correspondantes aux p-values faibles. D'abord ces p-values sont dépendantes du contexte, c'est-à-dire  de l'ensemble des variables considérées dans le modèle. Voir la p-value de Newspaper seul, et de Newspaper avec Radio et TV. Ensuite, lorsqu'on a beaucoup de variables, il est très possible d'obtenir des p-values faibles juste par chance (ou par malchance !) alors que les variables considérées ne sont pas explicatives. 

La F-statistique ne souffre pas de ce défaut car elle prend en compte le nombre total de prédicteurs.  L'idée va alors être de comparer cette F-statistique pour tous les modèles possibles deux à deux. Le souci, c'est que pour $p$ variables, il y a $2^p$ modèles possibles... cela devient donc très rapidement impratiquable. Il existe différentes stratégies qui permettent de réduire le nombre de modèles à considérer, il s'agit de Forward Selection, Backward Selection et Mixed Selection. Partant de ces modèles, différents critères statistiques Cp, AIC, BIC, adjusted R2 peuvent être utilisés, qui combinent généralement le R2 avec un pénalisation de la dimension du modèle. On pourra également utiliser une stucture de *validation croisée*. À suivre.

## Effets de synergie - Prise en compte de non-linéarités


### Synergies

Revenons à la représentation du plan de prédiction pour les deux variables explicatives TV et Radio. 

In [ ]:
%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

# Create plot
fig = plt.figure(figsize=(8,6))
fig.suptitle('Regression: Sales ~ Radio + TV Advertising', fontsize=20)

ax = fig.add_subplot(111, projection='3d')

x = advertising.Radio
y = advertising.TV
z = advertising.Sales

ax.scatter(x, y, z, c='r', marker='o')
ax.plot_surface(B1, B2, Z, rstride=10, cstride=5, alpha=0.4)

ax.set_xlabel('Radio')
ax.set_xlim(0,50)
ax.set_ylabel('TV')
ax.set_ylim(ymin=0)
ax.set_zlabel('Sales')

plt.show()

In [ ]:
azim=widgets.FloatSlider(min=-180, max=180, step=1, value=-60)    
elev=widgets.FloatSlider(min=-180, max=360, step=1, value=30)   

#ax.set_axis_off()
@interact(elev=elev, azim=azim)
def view(elev, azim):
    ax.view_init(elev, azim)
    #ax.azim=azim
    #ax.elev=elev
    display(ax.figure)

Sur cette figure, il semble que l'on ait une déviation à la linéarité. En effet lorsque le budget est surtout affecté à l'un des pédicteurs, on sur-estime les ventes, alors que lorsque le budget est réparti, on a plutôt tendance à sous-estimer. Ceci suggère un comportment non-linéaire et même une *synergie* ou une *interaction* entre les prédicteurs...

In [ ]:
ax.view_init(elev=26, azim=36)
display(ax.figure)

Le modèle linéaire que nous avons considéré juqu'ici *découple* l'effet des variables entre-elles. En d'autres termes, on considère que l'effet d'ajouter du budget sur le poste TV est complètement indépendant du budget déjà engagé sur radio (et réciproquement). Ceci n'est pas nécessairement exact. En effet, il est possible qu'investir sur l'un des postes infuence le rendement du second. Par exemple, il est possible qu'investir sur la radio augmente l'effet d'une dépense sur la TV (la pente pour TV augmente avec Radio). En marketing, ceci est appelé un effet de synergie. 
Avec le modèle de départ,
$$
Sales \sim \beta_0 + \beta_1 TV + \beta_2 Radio,
$$
lorsqu'on augmente TV d'une unité, Sales augmente de $\beta_1$, quelque soit le niveau de Radio.

Une manière de modifier ce comportement pour rendre compte d'une éventuelle synergie est d'introduire un prédicteur pour un terme d'interaction sous la forme TV*Radio : 
$$
Sales \sim \beta_0 + \beta_1 TV + \beta_2 Radio + \beta_3 TV.Radio.
$$
L'effet se comprend aisément en ré-écrivant cette relation sous la forme
$$
Sales \sim \beta_0 + (\beta_1+\beta_3 Radio) TV + \beta_2 Radio,
$$
qui montre bien que cette fois le coefficient de TV est directement lié à Radio. 

Avec R, il n'y a qu'à demander : 

- Pour rappel les résultats sans ce terme :

In [ ]:
%%R
lmModel = lm(Sales ~ Radio+TV, data=advertising)
summary(lmModel)

- Et avec le terme de synergie :

In [ ]:
%%R
lmModel = lm(Sales ~ Radio+TV+Radio*TV, data=advertising)
summary(lmModel)

On constate donc que la RSE passe de 1.681 à 0.9435, et que le terme de synergie présente une $p$-value extrêmement faible. Ceci indique donc à la fois l'existence d'une synergie, la pertinence de ce terme, et l'effet très important en terme de réduction de la RSE. 

En R, il est encore possible de spécifier les termes d'interaction par une formulation comme

In [ ]:
%%R
lmModel = lm(Sales ~ (Radio+TV)^2, data=advertising)
summary(lmModel)

### Non linéarités

In [ ]:
##%%R -o auto
#auto=read.csv(file = "http://www-bcf.usc.edu/~gareth/ISL/Auto.csv")
#summary(auto)

In [ ]:
##%%R 
#write.csv(auto, file = "Auto.csv", row.names=FALSE)

In [ ]:
%%R
auto = read.csv("Auto.csv",na.strings = "?")
auto = na.omit(auto)
summary(auto)

On va chercher à prédire mpg (*gas mileage in miles per gallon*) en fonction de la puissance *horsepower* :

In [ ]:
%%R
lm.fit = lm(mpg ~ horsepower, data=auto)
summary(lm.fit)
plot(auto$horsepower, auto$mpg, col = "blue", pch = 4)
abline(lm.fit, lwd = 3, col = "red")
plot(lm.fit, which=1)

In [ ]:
%%R
lm.fit = lm(mpg ~ horsepower + I(horsepower^2), data=auto)
#
plot(auto$horsepower, auto$mpg, col = "blue", pch = 4)
minMax = range(auto$horsepower)
xvals = seq(minMax[1], minMax[2], len = 100) 
# Use predict based on a dataframe containing 'horsepower'
yvals = predict(lm.fit, newdata = data.frame(horsepower=xvals))
lines(xvals, yvals,type="l", col="red", lw=2)
#plot(lm.fit, which=1)
summary(lm.fit)


A un ordre plus élevé, on peut utiliser la fonction `poly` :


In [ ]:
%%R
lm.fit = lm(mpg ~ poly(horsepower,5), data=auto)
#
plot(auto$horsepower, auto$mpg, col = "blue", pch = 4)
minMax = range(auto$horsepower)
xvals = seq(minMax[1], minMax[2], len = 100) 
# Use predict based on a dataframe containing 'horsepower'
yvals = predict(lm.fit, newdata = data.frame(horsepower=xvals))
lines(xvals, yvals,type="l", col="red", lw=2)
#plot(lm.fit, which=1)
print(paste("R2=",summary(lm.fit)$r.sq))

In [ ]:
%%R
lm.fit = lm(mpg ~ (horsepower+weight+acceleration)^3, data=auto)
#
summary(lm.fit)

In [ ]:
# RESTART KERNEL HERE

import matplotlib.pyplot as plt
from IPython.display import clear_output, display, HTML, Image, Javascript
from ipywidgets import interactive, fixed
import ipywidgets as widgets
import numpy as np
import pandas as pd

%load_ext rpy2.ipython

En Python, en utilisant `seaborn`

In [ ]:
import seaborn as sns
auto = pd.read_csv("Auto.csv",na_values="?").dropna()

plt.scatter(auto.horsepower, auto.mpg, facecolors='None', edgecolors='k') 
sns.regplot(x=auto.horsepower, y=auto.mpg, ci=None, label='Linear', scatter=False)
sns.regplot(x=auto.horsepower, y=auto.mpg, ci=None, label='Degree 2', order=2, scatter=False)
sns.regplot(x=auto.horsepower, y=auto.mpg, ci=None, label='Degree 5', order=5, scatter=False)
plt.legend()
plt.ylim(5,55)
plt.xlim(40,240);

## Cas des variables catégorielles

Il est tout-à-fait possible de gérer des variables catégorielles (des facteurs). Pour cela, on crée des *dummy variables*  binaires qui couvrent tous les niveaux de la variable catégorielle initiale. Ainsi, si on a une variable numérique $X_1$ et une seconde variable $X_2$ à $L$ niveaux, on crée $p=\lceil\log_2 L\rceil$ variables $X_{2i}$ et le modèle devient
$$
Y = \beta_0 + \beta_1 X_1 + \sum_{i=1}^p \beta_{2i} X_{2i} + \epsilon
$$


## Diagnostic - Residual plot

L'hypothèse initiale utilisée pour bâtir le modèle est une hypothèse de linéarité et d'erreur de variance constante
$$
\displaystyle{
Y = \beta_0 + \beta_1 X_1 + \ldots \beta_p X_p + \epsilon.
}
$$
Afin de tester ces hypothèses, on peut soit utiliser une approche par tests d'hypothèse, soit une représentation dignostic. Parmi les représentations disponibles, le *residual plot* représente les résidus obtenus en fonction des valeurs prédites (*fitted*)

### Hétéroscedasticité, non-linéarité

Voici quelques exemples de représentation qui peuvent intervenir (sur un exemple tiré de [Faraway: Linear Models with R]  (2005, p. 59)).
\begin{figure}[H]
\centerline{\includegraphics[width=10cm]{residuals_example.png}}
\caption{\label{fig:example_residual} Exemples de residual plots.}
\end{figure}
![](residuals_example.png)

Sur l'exemple des données de publicité, on a:

In [ ]:
%%R
advertising = read.csv(file = "Advertising.csv")
lmModel = lm(Sales ~ TV, data=advertising)
summary(lmModel)

In [ ]:
%%R
plot(lmModel, which=1)

soit une indication d'un peu d'hétéroscedasticité (qui peut être éventuellement gérée en prenant $\log(Y)$ ou $\sqrt{Y}$). 

In [ ]:
%%R
lmModel = lm(log(Sales) ~ TV, data=advertising)
plot(lmModel, which=1)
summary(lmModel)

### Données aberrantes - *outliers*

Les outliers sont des données aberrantes, typiquement liées à un problème d'enregistrement (erreur sur un bit) par exemple. Elles présentent généralement une différence d'amplitude par rapport aux autres données. Une manière de détecter les outliers est de comparer les résidus à leur écart-type -- le rapport est alors le *studentized residual* (rapport d'une gaussienne à chi2) et de rejeter les rapports supérieurs à 3 en module. 

Une illustration issue de  [An Introduction to Statistical Learning
with Applications in R](http://www-bcf.usc.edu/~gareth/ISL/) : 
\begin{figure}[H]
\centerline{\includegraphics[width=10cm]{example_studentized_residuals.png}}
\caption{\label{fig:example_residual} Exemples de studentized residual.}
\end{figure}
![](example_studentized_residuals.png)

Voici une manière de les obtenir en R : 

In [ ]:
%%R
#studentized residual - typiquement, les valeurs doivent être comprises entre -3 et 3
plot(fitted(lmModel),rstudent(lmModel))

### High leverage 

Le [*leverage*](https://en.wikipedia.org/wiki/Leverage_%28statistics%29) est une mesure de l'influence d'une variable particulière sur la régression. Voici une illustration où on ajoute un simple point qui a une influence non négligeable sur la régression. 

In [ ]:
%%R
set.seed(20)

x1 = rnorm(50, mean=20, sd=3)
y1 = 5 + .5*x1 + rnorm(20)

x4 = c(x1, 30);        y4 = c(y1, 10)
lm1=lm(y1~x1)
lm2=lm(y4~x4)
plot(y4~x4)
points(30,10,pch=23, col="red")
abline(lm1,col="blue")
abline(lm2,col="red")

Le leverage représente l'influence de la donnée sur sa prédiction et est défini par
$$
h_{ii} = \frac{\partial \hat{y}_i}{y_i}.
$$
On montre qu'il s'agit simplement du terme diagonal de la *hat-matrix* $H$ (\ref{eq:hatmatrix}). 

Les données à fort leverage peuvent être détectées en calculant le distance de Cook, ou dans le plan (leverage, studentized residuals) qui est le 5e diagnostic fourni par `plot.lm(model)`. Le leverage peut être obtenu par `hatvalues(model)`

In [ ]:
%%R
plot(lm1, which=5)
plot(lm2, which=5)

### Colinéarité

Lorsque des variables explicatives sont corrélées, il est difficile de séparer l'influence des unes et des autres sur la variable dépendante. Ceci se traduit par une augmentation de la variance du résultat, donc du RSE, des p-values, etc. Bref, c'est néfaste. Cela peut aussi se comprendre en constatant que la corrélation entre les colonnes de $\Xb$ va diminuer le conditionnement de $X^TX$ qui va devenir plus difficile à inverser. 
Pour juger de la colinéarité, on peut bien entendu examiner la matrice de corrélation. Ceci permet de repérer les variables franchement corrélées. Cependant, il peut aussi y avoir colinéarité lorsque plusieurs variables, 3, 4, 5 ou plus, sont corrélées entre elles ; dans ce cas, l'inspection directe de la matrice de corrélation est insuffisante. La stratégie consiste alors à essayer de faire une régression de chacune des variables en fonction de toutes les autres. On note ainsi $R^2_{X_j|X_{-j}}$ le R2 obtenu en effectuant la régression de $X_j$ vis-à-vis de l'ensemble des autres variables. Quand il y a colinéarité, ce dernier R2 est proche de 1. On définit alors un facteur le *VIF*, pour *Variance Inflation Factor*, par
$$ \label{eq:vif}
VIF(\hat{\beta_j}) = \frac{1}{1-R^2_{X_j|X_{-j}}}
$$
Sous R, on a directement accès à ce vif par `vif(model)` -- voir ce [post](https://beckmw.wordpress.com/2013/02/05/collinearity-and-stepwise-vif-selection/) à ce sujet. 

In [ ]:
%%R 
library(car)
advertising$corrvar = advertising$TV+6*advertising$Newspaper + rnorm(length(advertising$TV)) 
lmModel = lm(Sales ~ TV+Radio+Newspaper+corrvar, data=advertising)
vif(lmModel)

In [ ]:
%%R
cor(advertising)

Lorqu'on détecte un VIF important, il faut alors rechercher soit à (i) supprimer les variables redondantes (ii) combiner les variables redondantes entre elles en une nouvelle variable. 